In [2]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


# First lets look at the GOS

In [3]:
# %%timeit -r 1
nviews = 493 #493 #247 #165 #124 for the others

s = fc.Spectrum()
# s.load('Varian_truebeam')
s.load('W_spectrum_6')
s.x[0] = 1; s.x[1] = 2
s.attenuate(0.,fc.get_mu(z=13))

kernel = fc.Kernel(s,'CuGOS-784-micrometer')
kernel.add_focal_spot(1.2)
angles = np.linspace(0,np.pi*2,nviews)

fudge = 0.22/1.07765*1.3907
dose = fudge*3000/505

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
# phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,mgy=18,bowtie=True,filter='FF_t0') # should be 507

In [8]:
phantom.proj = phantom.proj * 1/0.97
phantom.reconstruct('FDK',filt='ram_lak')

In [10]:
phantom.img = (phantom.img[:,::2,::2] + phantom.img[:,1::2,::2] + phantom.img[:,::2,1::2] + phantom.img[:,1::2,1::2])/4
im = create_mask(phantom.img[5].shape,r=5.7,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[5],im)

## Now lets look at the bcca data

In [11]:
# np.save('no_noise_good_flat_filt_paper_draft_april',image_fc_scaled)
# image_fc_scaled = np.load('no_noise_good_flat_filt_paper_draft_april.npy')

In [22]:
plt.figure()
plt.imshow(phantom.img[5],cmap='gray')
plt.figure()
plt.imshow(10*recon_slice,cmap='gray')
plt.figure()
plt.imshow(np.hstack([1/0.97*phantom.img[5],recon_slice]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/xcite/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
bcca_recon = np.load('/home/xcite/MV_CBCT/old_recon_6x_300_rared.npy')
recon_slice = 10*np.rot90(np.mean(bcca_recon.T[31:52],0),2)
im = create_mask(recon_slice.shape,off =[-0.15,0], r= 5.75,radius=0.3)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in [-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04]:
    reses.append(minimize(HU_fun,[contrast_fc[0],contrast_fc[5]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
print(res)

x = [.853,0.945,.998,1.147,1.36,1.87]
contrast_fc_scaled = (contrast_fc - res.x[0])*(res.x[1])
image_fc_scaled = np.roll((phantom.img[4]- res.x[0] - 0.003)*(res.x[1]+0.004),5)

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.imshow(recon_slice,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(222)
plt.imshow(image_fc_scaled,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(223)
plt.plot(x,contrast_fc_scaled,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,contrast_bcca,'s',c='darkorange',label='Star-lack')
plt.subplot(224)
plt.plot(np.mean(recon_slice[250:260,7:-7],0),c='darkorange')
plt.plot(np.rot90(image_fc_scaled,0)[257,7:-7],c='cornflowerblue')
plt.tight_layout()

      fun: 0.03064262494444847
 hess_inv: array([[ 1.61510961e-04, -4.84532883e-04],
       [-4.84532883e-04,  5.06453599e-01]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 27
      nit: 4
     njev: 9
   status: 0
  success: True
        x: array([0.00926343, 1.14289781])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
np.save('fc_images/fc_image_gos_6x_493_small_noise',image_fc_scaled)

In [55]:
plt.imsave('im_for_imagej.png',np.hstack((recon_slice,image_fc_scaled)))
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]



# Now lets look at the kV data 

In [44]:
# %%timeit -r 1
nviews = 887 # 222 296 444 887 That is how many are in the file 
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(0.4,fc.get_mu(z=13)) #0.4
s.attenuate(0.089,fc.get_mu(z=22))
kernel = fc.Kernel(s,'CsI-784-micrometer')
kernel.add_focal_spot(2.2)
# angles = np.linspace(0.1,np.pi*2,nviews)        

(19, 19, 19) 19


In [45]:
angles = np.load('/home/xcite/MV_CBCT/angles_kv.npy')[::1]

In [51]:
conversion = 1.07765#*(0.511/0.784)**2

fudge = 0.3 # 0.018
# !!!!! Scatter is off
phantom = fc.Catphan_404()#pitch = 0.511)
# phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
# phantom.phan_map = ['air','G4_POLYSTYRENE','G4_POLYVINYL_BUTYRAL','G4_POLYVINYL_BUTYRAL','CATPHAN_Delrin','G4_POLYVINYL_BUTYRAL','CATPHAN_Teflon_revised','air','CATPHAN_PMP','G4_POLYVINYL_BUTYRAL','CATPHAN_LDPE','G4_POLYVINYL_BUTYRAL','CATPHAN_Polystyrene','air','G4_POLYVINYL_BUTYRAL','air','CATPHAN_Teflon_revised','air','air','air','air'] 
# phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,scat_on=True,mgy=0.004,bowtie=True,filter='bowtie_asym') #bowtie8 1.077 conversion was april mgy=fudge*21.1/1000
phantom.reconstruct('FDK',filt='ram_lak')

bowtie is off False
Using the bowtie filter True
really doing it!
bowtie scatter
ratio is 118.71110502710788 number of photons 0.6657473411255314


In [52]:
phantom.img = (phantom.img[:,::2,::2] + phantom.img[:,1::2,::2] + phantom.img[:,::2,1::2] + phantom.img[:,1::2,1::2])/4
im = create_mask(phantom.img[4].shape,r=5.75,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[4],im)

In [53]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kV_full.npy').T #recon_kv_jan_2021.npy')
recon_slice = np.rot90(np.mean(bcca_recon.T[32:33],0)).copy()
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in np.arange(-0.4,0.4,0.02):
    reses.append(minimize(HU_fun,[contrast_fc[2],contrast_fc[-3]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
    
fc_data = (contrast_fc - res.x[0])*(res.x[1])
# fc_image = np.roll((phantom.img[4] - (res.x[0]+0.0103))*(res.x[1]+0.0067),-4)
fc_image = np.roll((phantom.img[4] - (res.x[0]))*(res.x[1]),-4)

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
dist = np.linspace(-7.89,7.89,512-20)
dist2= np.linspace(-8,8,512-20)
plt.plot(dist,np.mean(recon_slice[250:260,10:-10],0),c='darkorange')
plt.plot(dist2,np.mean(np.rot90(np.roll(fc_image,7),2)[250:260,10:-10],0),c='cornflowerblue')
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
np.save('fc_images/fc_image_kv_887_noise',fc_image)

In [341]:
plt.imsave('data/im_for_imagej.png',np.hstack((recon_slice,fc_image)),cmap='gray')
!imagej data/im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]

Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: no element found
Fontconfig error: "/home/xcite/.config/fontconfig/fonts.conf", line 5: invalid attribute 'target'


# Now let's take a look at the images in terms of CNR

In [45]:
from glob import glob

In [46]:
im = create_mask([512,512],r=5.7,radius=0.4)
im2 = create_mask([512,512],off =[-0.15,0], r= 5.75,radius=0.3)

CNRs = []

plt.figure()

exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_6x_300_*.npy'))[np.array([0,1,3,2])]

ii = 0
for file_fc, file_exp in zip(sorted(glob('fc_images/fc_image_gos*.npy'))[-1::-1],exp_files):
    
    im_fc = np.load(file_fc)
    im_exp = np.rot90(np.load(file_exp).T[35],2)
    contrast_fc, CNR_fc, noise_fc = return_CNR(np.roll(im_fc,-5),im)
    contrast_bcca, CNR_exp, noise_bcca = return_CNR(im_exp,im2)#,show_map=True)
    CNRs.append(CNR_fc)
    plt.plot(ii,np.mean(CNR_fc),'b*',label=file_fc.split('/')[-1][-7:-4] + ' fc')
    plt.plot(ii+0.1,2.6*np.mean(CNR_exp),'r*',label=file_fc.split('/')[-1][-7:-4] +' exp')
    
    ii += 1

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Looking at the kV

In [80]:
im2 = create_mask([512,512],r=5.8,off = [0.2,0.05],rot = 1)
im = create_mask([512,512],r=5.75,radius=0.4)

CNRs = []

plt.figure()

exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_kV*.npy'))[np.array([0,3,2,1])]

ii = 0
for file_fc, file_exp in zip(sorted(glob('fc_images/fc_image_kv*.npy'))[-1::-1],exp_files):
    
    im_fc = np.load(file_fc)
    im_exp = np.rot90(np.mean(np.load(file_exp)[35:37],0),2)
    contrast_fc, CNR_fc, noise_fc = return_CNR(np.roll(im_fc,-5),im)
    contrast_bcca, CNR_exp, noise_bcca = return_CNR(im_exp,im2)#,show_map=True)
    CNRs.append(CNR_fc)
    plt.plot(ii,np.mean(CNR_fc),'b*',label=file_fc.split('/')[-1][-7:-4] + ' fc')
    plt.plot(ii+0.1,2.6*np.mean(CNR_exp),'r*',label=file_fc.split('/')[-1][-7:-4] +' exp')
    
    ii += 1

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
im_exp.shape,im2.shape

((512, 100), (512, 512))

# Now lets look at the 2.5 MV

In [2]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [40]:
# %%timeit -r 1
nviews =  493 #493 #247 #165 #124 for the others

s = fc.Spectrum()
s.load('W_spectrum_25')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
# kernel.add_focal_spot(1.2)
angles = np.load('/home/xcite/MV_CBCT/angles_25x.npy')

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic',
                    'CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised',
                    'air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,mgy=600*3/nviews,bowtie=True,filter='FF0') # should be 507
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[5].shape,r=5.7,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[5],im,show_map=True)

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 17874.726480268397 number of photons 0.01015508406165716


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Now lets look at the bcca data

In [41]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_25x_60.npy')
recon_slice = np.rot90(bcca_recon.T[20],2)
im = create_mask(recon_slice.shape,off =[0.15,0], r= 5.75,radius=0.3)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

In [42]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in [-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04]:
    reses.append(minimize(HU_fun,[contrast_fc[0],contrast_fc[5]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]

x = [.853,0.945,.998,1.147,1.36,1.87]
contrast_fc_scaled = (contrast_fc - res.x[0])*(res.x[1])
image_fc_scaled = np.roll((phantom.img[4]- res.x[0])*(res.x[1]),5)

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.imshow(recon_slice,cmap='gray',vmin=0.003,vmax=0.014)
plt.axis('off')
plt.subplot(222)
plt.imshow(image_fc_scaled,cmap='gray',vmin=0.003,vmax=0.014)
plt.axis('off')
plt.subplot(223)
plt.plot(x,contrast_fc_scaled,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,contrast_bcca,'s',c='darkorange',label='Star-lack')
plt.subplot(224)
plt.plot(recon_slice[256,7:-7])
plt.plot(np.rot90(image_fc_scaled,0)[257,7:-7])
plt.tight_layout()

/home/xcite/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now let's take a look at the images in terms of CNR

In [2]:
from glob import glob

In [105]:
im = create_mask([512,512],r=5.7,radius=0.4)
im2 = create_mask([512,512],off =[-0.15,0], r= 5.75,radius=0.3)

CNRs = []

plt.figure()

exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_6x_300_*.npy'))[np.array([0,1,3,2])]

ii = 0
for file_fc, file_exp in zip(sorted(glob('fc_images/fc_image_gos*.npy'))[-1::-1],exp_files):
    
    im_fc = np.load(file_fc)
    im_exp = np.rot90(np.load(file_exp).T[35],2)
                             
    contrast_fc, CNR_fc, noise_fc = return_CNR(np.roll(im_fc,-5),im)
    contrast_bcca, CNR_exp, noise_bcca = return_CNR(im_exp,im2)
                             
    CNRs.append(CNR_fc)
    plt.plot(ii,np.mean(CNR_fc),'b*',label=file_fc.split('/')[-1][-7:-4] + ' fc')
    plt.plot(ii+0.1,2.6*np.mean(CNR_exp),'r*',label=file_fc.split('/')[-1][-7:-4] +' exp')
    
    ii += 1

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:

no = 0.784
npt = 512

np2 = 512
no2 = 0.511

pts2 = np.linspace(0,np2*no2,np2) - np2/2*no2
pts = np.linspace(0,npt*no,npt) - npt/2*no

p2 = np.zeros_like(phantom.fs)
for ii in range(phantom.fs.shape[0]):
    p2[ii] = np.interp(pts2,pts,phantom.fs[ii])

plt.figure()
plt.plot(pts,phantom.fs.T)
plt.plot(pts2,p2.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [39]:
len(p2.shape)

2

In [40]:
kernel.pitch

0.784